<a href="https://colab.research.google.com/github/aliyyah-u/NLP_Medical_NER/blob/main/NLP_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
__author__ = "Aliyyah U."
__version__ = "IN3045 City St George's, University of London, Spring 2025"

# Load Dataset

In [ ]:
!pip install --upgrade huggingface_hub

# Login to access dataset
from huggingface_hub import login
login()

import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_parquet("hf://datasets/parsa-mhmdi/Medical_NER/data/train-00000-of-00001.parquet")

train_indices, test_indices = train_test_split(df.index, test_size=0.2, random_state=42)

train_data = df.loc[train_indices]
test_data = df.loc[test_indices]

# Preview samples from train and test
print("Train sample:")
print(train_data.iloc[0])
print("\nTest sample:")
print(test_data.iloc[0])

# Token and tag lists for training set
train_tokens = train_data.tokens.tolist()
train_ner_tags = train_data.ner_tags.tolist()

# Preview
print("\nSample training tokens:")
print(train_tokens[0])
print("\nSample training tags:")
print(train_ner_tags[0])

# Extract unique tags
#train_unique_tags = set(tag for sublist in train_ner_tags for tag in sublist)
#print("\nUnique training NER tags:")
#print(sorted(train_unique_tags))

# Check if dataset was loaded
#df[:5]

#tokens = df.tokens.tolist()
#print (tokens[:1])

#extracting unique list of ner_tags
#ner_tags = df.ner_tags.tolist()
#print (ner_tags[:1])

#unique_tags = set(tag for sublist in ner_tags for tag in sublist)
#print(sorted(unique_tags))

In [ ]:
import spacy